In [ ]:
"""
MongoDB es una base de datos open source NoSql (Not only Sql) enfocada a manejar enormes cantidades de datos que se generan 
demasiado rápido (big data), por ejemplo los datos que vienen de aplicaciones de redes sociales, *juegos online o análisis de datos de múltiples sitios. 
Es flexible, escalable (podemos añadir mas recursos a nuestra base de datos añadiendo mas servidores - cluster-) y de propósito general (puede ser utilizada en cualquier tipo de aplicación).
Está orientada a DOCUMENTOS, que refiere a los datos que vamos a guardar en MongoDB (objetos de javascript).Utiliza JSON como formato de datos.
Las datos se organizan en COLECCIONES (por ej., una colección de usuarios, otra de productos, etc; en SQL serían las tablas). 
"""

In [8]:
#Imports
import pymongo
import pandas as pd

In [9]:
#Conexión a MongoDB
client = pymongo.MongoClient('localhost',27017)
db = client['mongo']

In [10]:
#Colección
collection = db['income']

In [11]:
#Lee un archivo, 
#Para que? 
#Donde está este archivo? 
#Como proceder para continuar con el programa??

with open('income_header.txt') as finput:
    column_names = finput.readline()
column_names

'age, workclass, fnlwgt, education, education-num, marital-status, occupation, relationship, race, sex, capital-gain, capital-loss, hours-per-week, native-country, label'

In [12]:
column_names_list = column_names.split(',')
column_names_list

['age',
 ' workclass',
 ' fnlwgt',
 ' education',
 ' education-num',
 ' marital-status',
 ' occupation',
 ' relationship',
 ' race',
 ' sex',
 ' capital-gain',
 ' capital-loss',
 ' hours-per-week',
 ' native-country',
 ' label']

In [13]:
#with open('income.txt')as finput:
with open('adult.data')as finput:
    for line in finput:
        row_list =line.rstrip('\n').split(',')
        row_dict = dict(zip(column_names_list,row_list))
        try:
            row_dict['age']= int(row_dict['age'])
            collection.insert_one(row_dict)
        except:
            pass

In [14]:
collection.count()

<ipython-input-14-7c4bc5758cb0>:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  collection.count()


32561

In [15]:
age39 = collection.find_one({'age':{'$eq' : 39}}) #alternatively, {'age': 39} can be used
print(age39)

{'_id': ObjectId('617c781eadbeb9dc2c50de57'), 'age': 39, ' workclass': ' State-gov', ' fnlwgt': ' 77516', ' education': ' Bachelors', ' education-num': ' 13', ' marital-status': ' Never-married', ' occupation': ' Adm-clerical', ' relationship': ' Not-in-family', ' race': ' White', ' sex': ' Male', ' capital-gain': ' 2174', ' capital-loss': ' 0', ' hours-per-week': ' 40', ' native-country': ' United-States', ' label': ' <=50K'}


In [16]:
ageover_45 = collection.find({'age':{"$gt" : 45}})
ageover_45.count()

<ipython-input-16-91897d8f5f17>:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  ageover_45.count()


9627

In [17]:
type(ageover_45)

pymongo.cursor.Cursor

In [18]:
ageover_45.next()

{'_id': ObjectId('617c781eadbeb9dc2c50de58'),
 'age': 50,
 ' workclass': ' Self-emp-not-inc',
 ' fnlwgt': ' 83311',
 ' education': ' Bachelors',
 ' education-num': ' 13',
 ' marital-status': ' Married-civ-spouse',
 ' occupation': ' Exec-managerial',
 ' relationship': ' Husband',
 ' race': ' White',
 ' sex': ' Male',
 ' capital-gain': ' 0',
 ' capital-loss': ' 0',
 ' hours-per-week': ' 13',
 ' native-country': ' United-States',
 ' label': ' <=50K'}

In [ ]:
"""
UPDATE
"""

In [19]:
collection.update_one(
        {"age": 38},
        {
        "$set": {
            "capital-gain":999,
            "capital-loss":999,
                }
        })

In [20]:
updated = collection.find_one({'age':{'$eq' : 38}})
updated

{'_id': ObjectId('617c781eadbeb9dc2c50de59'),
 'age': 38,
 ' workclass': ' Private',
 ' fnlwgt': ' 215646',
 ' education': ' HS-grad',
 ' education-num': ' 9',
 ' marital-status': ' Divorced',
 ' occupation': ' Handlers-cleaners',
 ' relationship': ' Not-in-family',
 ' race': ' White',
 ' sex': ' Male',
 ' capital-gain': ' 0',
 ' capital-loss': ' 0',
 ' hours-per-week': ' 40',
 ' native-country': ' United-States',
 ' label': ' <=50K',
 'capital-gain': 999,
 'capital-loss': 999}

In [ ]:
"""
Indexing in MongoDB
Primero busquemos cuánto tiempo lleva consultar algo usando find () como arriba sin un índice, 
así que cree una variable llamada start para realizar un seguimiento del tiempo, y llame a la función time 
en el paquete time en python
"""

In [21]:
import time
start = time.time()
ageover50 = collection.find({'age':{'$gt':50}})
end = time.time()
print(end-start)

0.0009996891021728516


In [22]:
#Creemos ahora un índice usando profiles.create_index ()
index_result = db.profiles.create_index([('age',pymongo.ASCENDING)],unique = False)

In [26]:
import time
start = time.time()
ageover45 = collection.find({'age':{'$gt':45}})
end = time.time()
print(end-start)
#La reducción de tiempo, incluso en una cantidad pequeña, es muy significativa. Por eso es importante la indexación.

0.0


In [ ]:
"""
Read data from MongoDB into dataframes

Convertimos el cursor, collection.find (), en una lista y luego en un dataframe de Pandas usando .DataFrame ()
"""

In [27]:
income_df = pd.DataFrame(list(collection.find()))

In [28]:
income_df.head()

,_id,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label,capital-gain,capital-loss
0,617c781eadbeb9dc2c50de57,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,NaN,NaN
1,617c781eadbeb9dc2c50de58,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,NaN,NaN
2,617c781eadbeb9dc2c50de59,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,999.0,999.0
3,617c781eadbeb9dc2c50de5a,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,NaN,NaN
4,617c781eadbeb9dc2c50de5b,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,NaN,NaN


In [29]:
income_df['age'].describe()
#El dataframe ahora se puede usar para realizar todas las tareas de ciencia de datos y aprendizaje automático

count    32561.000000
mean        38.581647
std         13.640433
min         17.000000
25%         28.000000
50%         37.000000
75%         48.000000
max         90.000000
Name: age, dtype: float64

In [ ]:
"""
delete ( ) in MongoDB

La colección creada anteriormente se puede eliminar usando delete_many () o selectivamente usando delete_one ()
"""

In [30]:
collection.delete_many({})